In [1]:
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end
includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### Example of Water Regulation 

In this notebook we exemplify the day-to-day activities of the water regulation company.  
For that we generate 1-2 simple reservoir systems, in which power producers nominate for discharge at different reservoirs.  
Concepts such as the adjusted flow, power swap, reduction through spillage should be readily apparent after working through the examples.  
We also visualize what happens if we change our nomination: When does spillage happen, how does the adjusted flow change, how does it affect the power swap, how does it affect the amount of power we can produce?  
Something else to consider is how we can calculate the reference flow. Maybe do this in this file, or in another file where we work more closely with the real water data.

In [6]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
R, K, J = read_data(filepath_Ljungan)
print() 

j = J[1]
K_j = j.plants
O, K_O = OtherParticipant(J, j, R)
pj = j.participationrate
pO = O.participationrate


println("Participation rate $(j.name): \n ", pj)
println("Participation rate $(O.name): \n ", pO)
println("K_O : $(K_O) and spillage $(K_O[1].spillreference),\nK_j : $(K_j) and spillage $(K_j[1].spillreference)")

Participation rate Sydkraft: 
 ________________________________
Holsmjon | 0.0     
Flasjon  | 1.84    

Participation rate Other: 
 ________________________________
Holsmjon | 2.68    
Flasjon  | 2.68    

K_O : HydropowerPlant[Parteboda, Hermansboda, Ljunga, Nederede, Skallbole, Matfors, Viforsen, Jarnvagsforsen] and spillage 1.4,
K_j : HydropowerPlant[Flasjo, Trangfors, Ratan, Turinge, Bursnas] and spillage 0.58


In [16]:
filepath_Simple = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\SimpleReservoirSystem.json"
R1, K1, J1 = read_data(filepath_Simple)

println("The reservoir system is made up of the following components: \n ______________________________________________________")
println("Reservoirs: ", [r.dischargepoint for r in R1])
for r in R1
    println("The reservoir $(r.dischargepoint) has a maximum capacity of $(r.maxvolume).")
end
println("Power Plants: ", [k.name for k in K1])
println("The power plants have the following production capacities until there is spillage: ")
for k in K1
    println("$(k.name) has spillage upper bound of $(k.spillreference). Additionally, the equivalent is $(k.equivalent), thus the maximum power is $(k.equivalent * k.spillreference)")
end
println("Participants: ", [p.name for p in J1])
println("The Participants are owners of the following power plants: ")
for j in J1
    println("$(j.name) is owner of $([k.name for k in j.plants]). Because of this ownership the participationrate becomes \n $(j.participationrate)")
end

The reservoir system is made up of the following components: 
 ______________________________________________________


Reservoirs: ["A"]
The reservoir A has a maximum capacity of 9000.0.
Power Plants: ["Kraftwerk1", "Kraftwerk2", "Kraftwerk3"]
The power plants have the following production capacities until there is spillage: 
Kraftwerk1 has spillage upper bound of 1.0. Additionally, the equivalent is 0.4, thus the maximum power is 0.4
Kraftwerk2 has spillage upper bound of 1.2. Additionally, the equivalent is 0.7, thus the maximum power is 0.84
Kraftwerk3 has spillage upper bound of 1.3. Additionally, the equivalent is 0.3, thus the maximum power is 0.39


Participants: ["EON", "RWE", "Statkraft"]
The Participants are owners of the following power plants: 
EON is owner of ["Kraftwerk1"]. Because of this ownership the participationrate becomes 
 ________________________________
A        | 0.4     

RWE is owner of ["Kraftwerk2"]. Because of this ownership the participationrate becomes 
 ________________________________
A        | 0.7     

Statkraft is owner of ["Kraftwerk3"]. Because of this ownership the participationrate becomes 
 ________________________________
A        | 0.3     

